# Análise de Portfólio
Análise de portfólio usando a Fronteira Eficiente de Markowitz e otimização por simulações.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
Pesos = {"ITUB4.SA":0.10, "CPLE3.SA":0.20, "LREN3.SA":0.20, "EMBR3.SA":0.10, "PSSA3.SA":0.10, "GOAU4.SA":0.30}

Periodo_Inicio = datetime.datetime(2016,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

In [ ]:
DadosAcoes = pdr.get_data_yahoo(list(Pesos.keys()), start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados = DadosAcoes["Adj Close"]

In [ ]:
Dados

### Fronteira Eficiente de Markowitz
Calcula o retorno esperado para um portfólio e cria fronteira eficiente simulando 10.000 portfólios.

- Retorno dos Ativos

In [ ]:
RetornoDosAtivos = Dados.pct_change().fillna(0)

In [ ]:
RetornoDosAtivos

- Retorno Médio Diário Esperado

In [ ]:
RetornoMedioDiario = RetornoDosAtivos.mean()

In [ ]:
RetornoMedioDiario

- Retorno Médio Mensal Esperado

In [ ]:
RetornoMedioMensal = Dados.resample("M").last().pct_change().mean()

In [ ]:
RetornoMedioMensal

- Retorno Médio Anualizado Esperado

In [ ]:
RetornoMedioAnualizado = Dados.resample("Y").last().pct_change().mean()

In [ ]:
RetornoMedioAnualizado

- Matriz de Covariância

In [ ]:
CovarianciaDosRetornos = RetornoDosAtivos.cov()

In [ ]:
CovarianciaDosRetornos

- Matriz de Correlação

In [ ]:
CorrelacaoDosRetornos = RetornoDosAtivos.corr()

In [ ]:
CorrelacaoDosRetornos

- Retornos Esperados do Portfólio

In [ ]:
RetornoMedioDiarioDoPortfolio = (list(Pesos.values()) * RetornoMedioDiario).sum()
RetornoMedioMensalDoPortfolio = (list(Pesos.values()) * RetornoMedioMensal).sum()
RetornoMedioAnualizadoDoPortfolio = (list(Pesos.values()) * RetornoMedioAnualizado).sum()

In [ ]:
print("Retorno Diário do Portfólio: {0:.2%}".format(RetornoMedioDiarioDoPortfolio))
print("Retorno Mensal do Portfólio: {0:.2%}".format(RetornoMedioMensalDoPortfolio))
print("Retorno Anualizado do Portfólio: {0:.2%}".format(RetornoMedioAnualizadoDoPortfolio))

- Riscos do Portfólio

In [ ]:
MatrizRiscoDosAtivos = CovarianciaDosRetornos.mul(Pesos, axis=0).mul(Pesos, axis=1).sum().sum()

In [ ]:
RiscoDiario = np.sqrt(MatrizRiscoDosAtivos)
RiscoMensal = np.sqrt(MatrizRiscoDosAtivos) * np.sqrt(21)
RiscoAnualizado = np.sqrt(MatrizRiscoDosAtivos) * np.sqrt(252)

In [ ]:
print("Risco Diário do Portfólio: {0:.2%}".format(RiscoDiario))
print("Risco Mensal do Portfólio: {0:.2%}".format(RiscoMensal))
print("Risco Anualizado do Portfólio: {0:.2%}".format(RiscoAnualizado))

### Simulação de Portfólios
Simula um número de portfólios baseado nos seguintes parâmetros:
- Ativos: Dataframe de Preços de Ativos.
- NumPortfolios: Número de portfólios à simular.

In [ ]:
def SimularPortfolios(Ativos, NumPortfolios):
    RetornosSimulados = []
    RiscosSimulados = []
    PesosSimulados = []
    NumAtivos = len(Ativos.columns)

    # Retorno dos Ativos
    Ativos = Ativos.pct_change().fillna(0)
    MatrizCovariancia = RetornoDosAtivos.cov()
    
    for portfolio in range(NumPortfolios):

        # Simula pesos para um portfólio
        Pesos = np.random.random(NumAtivos)
        Pesos = Pesos/np.sum(Pesos)
        PesosSimulados.append(Pesos)
        
        # Retorno do portfolio simulado
        Retornos = Ativos.dot(Pesos).values
        Retorno = Retornos.mean()
        RetornosSimulados.append(Retorno)
        
        # Risco do portfólio simulado
        Variancias = MatrizCovariancia.mul(Pesos, axis=0).mul(Pesos, axis=1).sum().sum()
        Riscos = np.sqrt(Variancias)
        RiscosSimulados.append(Riscos)

    # Dataframe dos Portfólios simulados
    Simulacoes = {'Retorno':RetornosSimulados, 'Risco':RiscosSimulados}
    for counter, symbol in enumerate(Ativos.columns.tolist()):
        Simulacoes[symbol] = [w[counter] for w in PesosSimulados]
    
    Portfolios = pd.DataFrame(Simulacoes)
    return(Portfolios)

In [ ]:
Portfolios = SimularPortfolios(Dados, 10000)

In [ ]:
Portfolios

- Performance de cada Portfólio

In [ ]:
Portfolios["Sharpe"] = Portfolios["Retorno"] / Portfolios["Risco"]

In [ ]:
Portfolios

- Fronteira Eficiente Risco vs Retorno

In [ ]:
FronteiraEficiente = plg.Figure(plg.Scatter(x=Portfolios["Risco"],
                                            y=Portfolios["Retorno"],
                                            mode="markers", marker_symbol="circle", marker_opacity=0.5,
                                            name="Portfolios Simulados"))
FronteiraEficiente.update_layout(xaxis_tickformat=".2%", yaxis_tickformat=".2%")
Plot_Show(FronteiraEficiente, "Fronteira Eficiente")

- Portfólio de Menor Risco

In [ ]:
PortfolioMenorRisco = Portfolios.iloc[Portfolios["Risco"].idxmin()]

In [ ]:
PortfolioMenorRisco

- Portfólio de Maior Retorno

In [ ]:
PortfolioMaiorRetorno = Portfolios.iloc[Portfolios["Retorno"].idxmax()]

In [ ]:
PortfolioMaiorRetorno

- Portfólio de Melhor Performance

In [ ]:
PortfolioMelhorSharpe = Portfolios.iloc[Portfolios["Sharpe"].idxmax()]

In [ ]:
PortfolioMelhorSharpe

- Fronteira Eficiente com Melhores Portifólios 

In [ ]:
FronteiraEficiente = plg.Figure()

FronteiraEficiente.add_trace(plg.Scatter(x=Portfolios["Risco"],
                                         y=Portfolios["Retorno"],
                                         mode="markers", marker_symbol="circle", marker_opacity=0.5,
                                         name="Portfolios Simulados"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMenorRisco["Risco"]],
                                         y=[PortfolioMenorRisco["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor Risco"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMaiorRetorno["Risco"]],
                                         y=[PortfolioMaiorRetorno["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Retorno"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorSharpe["Risco"]],
                                         y=[PortfolioMelhorSharpe["Retorno"]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Sharpe"))
FronteiraEficiente.update_layout(xaxis_tickformat=".2%", yaxis_tickformat=".2%")
Plot_Show(FronteiraEficiente, "Fronteira Eficiente")